# Deploying ML Models through API

## Train and Save Model

In [1]:
import numpy as np 
import pandas as pd 
import sklearn.utils as skutils
import sklearn.model_selection as skmodelsel
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import requests
import io
url="https://manmohan291.s3.ap-south-1.amazonaws.com/LinearRegressionDS/CylinderVolumeData.zip"
s=requests.get(url).content
df=pd.read_csv(io.BytesIO(s),compression='zip')

df = skutils.shuffle(df)
dfTrain, dfValid = skmodelsel.train_test_split(df, test_size=0.2)

df_Features=dfTrain.loc[:,['radius','height']]
df_Label=dfTrain.loc[:,['volume']]
X=df_Features.values
Y=df_Label.values

degree=3

#Preprocessing
SS=StandardScaler()
XS=SS.fit_transform(X)
PF=PolynomialFeatures(degree)
XS_P=PF.fit_transform(XS)
#Train
RegObj=SGDRegressor(learning_rate='adaptive',eta0=0.1,max_iter=1000,alpha=0).fit(XS_P,Y.flatten())

#Save
import pickle
with open('my_ml_model.pkl', 'wb') as f:
    data={"regobj":RegObj,"SS":SS,"PF":PF}
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

# Load Model

In [2]:
from os import system
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from flask import Flask

In [3]:
import pickle
from sklearn.preprocessing import PolynomialFeatures

In [4]:
with open('my_ml_model.pkl', 'rb') as f:
    data= pickle.load(f)
RegObj=data['regobj']
SS=data['SS']
PF=data['PF']

# Host Model

In [5]:
#Predict
pX=[[189,177]]
pXS=SS.transform(pX)
pXS_P=PF.transform(pXS)
pY=RegObj.predict(pXS_P)
pY=pY
print(np.round(pY,2))

[19863087.12]


In [6]:
from flask import Flask 
 
app = Flask(__name__) # init Flask with current module as argument
  
# The route() function associates URL to function. 
@app.route('/') # ‘/’ URL is bound with hello_world() function. 
def Page1(): 
    return 'Welcome to Machine Learning APIs'


@app.route('/volume/<int:radius>/<int:height>')  
def Page2(radius,height): 
    X=np.array([[radius,height]])
    XS=SS.transform(X)
    pX=PF.transform(XS)
    pY=RegObj.predict(pX)
    htmlStr= "<font color=nevy><br>Radius is "+str(radius )
    htmlStr=htmlStr+ "<font color=nevy><br>Height is "+str(height)
    htmlStr=htmlStr+ "<font color=red><br>Volume is "+str(np.round(pY,2) )
    return htmlStr



# main function starts Application and keep it Running
if __name__ == '__main__': 
    app.run()#host='0.0.0.0',port="5080")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jan/2020 15:50:53] "GET /volume/189/177 HTTP/1.1" 200 -


# Run API URL

http://127.0.0.1:5000/volume/189/177